In [1]:
import math
import pandas as pd
import emoji
import torch
from torch import nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
import time
import math
import random
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
from datetime import datetime
import plotly.offline as pyo
from d2l import torch as d2l
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
df=pd.read_csv("tweets_df.csv")
df

,Datetime,Tweet Id,Text,Username
0,2021-10-18 08:53:00+00:00,1450022089218478082,"15,000,000 #XLM (5,798,314 USD) transferred fr...",whale_alert
1,2021-10-18 08:51:42+00:00,1450021762670940161,"🔥 5,219,354 #USDC (5,219,354 USD) burned at U...",whale_alert
2,2021-10-18 08:51:06+00:00,1450021611399168000,"🔥 5,538,816 #USDC (5,538,816 USD) burned at U...",whale_alert
3,2021-10-18 08:40:12+00:00,1450018866785132551,"🔥 5,730,249 #USDC (5,730,249 USD) burned at U...",whale_alert
4,2021-10-18 08:09:33+00:00,1450011152457535489,"💵 💵 💵 💵 💵 💵 💵 💵 💵 💵 157,091,936 #BUSD (157,09...",whale_alert
...,...,...,...,...
52782,2018-09-12 18:11:03+00:00,1039939462539763713,"15,720 BTC (99,365,473 USD) from Unknown (http...",whale_alert
52783,2018-09-12 17:26:18+00:00,1039928200061771777,"16,220 BTC (102,313,554 USD) from Unknown (htt...",whale_alert
52784,2018-09-12 17:09:32+00:00,1039923978729672713,"🚨 25,000 ETH (4,297,416 USD) to Exchange trans...",whale_alert
52785,2018-09-12 17:08:12+00:00,1039923645395685376,"🚨 25,000 ETH (4,286,963 USD) to Exchange trans...",whale_alert


In [3]:
df=df[df['Text'].str.contains(r'^(?=.*#BTC)')]

In [4]:
df["Text"] = df["Text"].replace(r'http\S+','', regex=True) #remove url
df["Text"] = df["Text"].replace(r'\n','', regex=True) # remove new lines
df["Text"]= df["Text"].replace(',','', regex=True) # remove commas
df["Text"]= df["Text"].replace('USD','', regex=True) #remove USD to allow taking of numbers in brackets
df["Text"]= df["Text"].replace('Tx:','', regex=True) # idk whats this
df["Text"]= df["Text"].replace('To:','', regex=True) #remove to

<ipython-input-4-304f354f6185>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-304f354f6185>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-304f354f6185>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-304f354f618

In [5]:
df["kw_check"]=df["Text"].str.extract(r'(\w*?)\s*\(')
df=df[df["kw_check"]=="BTC"]
df

<ipython-input-5-2c8869c85b47>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Datetime,Tweet Id,Text,Username,kw_check
12,2021-10-18 05:58:14+00:00,1449978106010349568,🚨 🚨 🚨 520 #BTC (32267027 ) transferred from #...,whale_alert,BTC
16,2021-10-18 05:07:52+00:00,1449965431041806339,🚨 🚨 🚨 525 #BTC (32724703 ) transferred from #...,whale_alert,BTC
18,2021-10-18 04:58:46+00:00,1449963142872768515,🚨 🚨 🚨 515 #BTC (32132338 ) transferred from #...,whale_alert,BTC
24,2021-10-17 21:34:18+00:00,1449851288083177472,🚨 🚨 400 #BTC (23953498 ) transferred from #Bi...,whale_alert,BTC
25,2021-10-17 21:34:18+00:00,1449851285948276743,🚨 🚨 🚨 645 #BTC (38674244 ) transferred from #...,whale_alert,BTC
...,...,...,...,...,...
52674,2018-09-19 09:04:06+00:00,1042338533606912000,955 #BTC (6083566 ) transferred from #Bitfinex...,whale_alert,BTC
52678,2018-09-19 04:45:51+00:00,1042273540760850434,946 #BTC (6012967 ) transferred from #Bitfinex...,whale_alert,BTC
52680,2018-09-19 00:06:55+00:00,1042203344012550144,1500 #BTC (9557771 ) transferred from Unknown ...,whale_alert,BTC
52682,2018-09-18 09:15:51+00:00,1041979103442812928,997 #BTC (6269280 ) transferred from #Bitfinex...,whale_alert,BTC


In [6]:
df['BTC']=df["Text"].str.extract(r'(\S+?) #BTC')

In [7]:
df['USD']=df["Text"].str.extract(r'\((.*?)\)')

In [8]:
df['USD']

12       32267027 
16       32724703 
18       32132338 
24       23953498 
25       38674244 
           ...    
52674     6083566 
52678     6012967 
52680     9557771 
52682     6269280 
52683     6287920 
Name: USD, Length: 10517, dtype: object

In [9]:
df['USD']=df['USD'].str.replace('[^0-9]', '', regex=True)
df['USD']

12       32267027
16       32724703
18       32132338
24       23953498
25       38674244
           ...   
52674     6083566
52678     6012967
52680     9557771
52682     6269280
52683     6287920
Name: USD, Length: 10517, dtype: object

In [10]:
#df['Source']=df["Text"].str.extract(r'(?<=from )(\w+)')
#df['Source']=df["Text"].str.extract(r'(\S+?) to')
df['Source']=df["Text"].str.extract(r"from(.*?)to")
df['Source']=df['Source'].str.lower()

In [11]:
df['Destination']=df["Text"].str.extract(r'(?<=to )(.*)')
df['Destination']=df['Destination'].str.lower()

In [12]:
df["transferred"]=df["Text"].str.extract(r'(\S+?) from')

In [13]:
df["scam"]=df["Text"].str.contains("scam")

In [14]:
df["stolen"]=df["Text"].str.contains("stolen")

In [15]:
df=df[(df['scam']==False)&(df['stolen']==False)]
df

,Datetime,Tweet Id,Text,Username,kw_check,BTC,USD,Source,Destination,transferred,scam,stolen
12,2021-10-18 05:58:14+00:00,1449978106010349568,🚨 🚨 🚨 520 #BTC (32267027 ) transferred from #...,whale_alert,BTC,520,32267027,#bitstamp,#binance,transferred,False,False
16,2021-10-18 05:07:52+00:00,1449965431041806339,🚨 🚨 🚨 525 #BTC (32724703 ) transferred from #...,whale_alert,BTC,525,32724703,#coinbase,#binance,transferred,False,False
18,2021-10-18 04:58:46+00:00,1449963142872768515,🚨 🚨 🚨 515 #BTC (32132338 ) transferred from #...,whale_alert,BTC,515,32132338,#coinbase,#binance,transferred,False,False
24,2021-10-17 21:34:18+00:00,1449851288083177472,🚨 🚨 400 #BTC (23953498 ) transferred from #Bi...,whale_alert,BTC,400,23953498,#binance,#bitstamp,transferred,False,False
25,2021-10-17 21:34:18+00:00,1449851285948276743,🚨 🚨 🚨 645 #BTC (38674244 ) transferred from #...,whale_alert,BTC,645,38674244,#coinbase,unknown wallet,transferred,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
52674,2018-09-19 09:04:06+00:00,1042338533606912000,955 #BTC (6083566 ) transferred from #Bitfinex...,whale_alert,BTC,955,6083566,#bitfinex,unknown wallet,transferred,False,False
52678,2018-09-19 04:45:51+00:00,1042273540760850434,946 #BTC (6012967 ) transferred from #Bitfinex...,whale_alert,BTC,946,6012967,#bitfinex,unknown wallet,transferred,False,False
52680,2018-09-19 00:06:55+00:00,1042203344012550144,1500 #BTC (9557771 ) transferred from Unknown ...,whale_alert,BTC,1500,9557771,unknown wallet,#bitfinex,transferred,False,False
52682,2018-09-18 09:15:51+00:00,1041979103442812928,997 #BTC (6269280 ) transferred from #Bitfinex...,whale_alert,BTC,997,6269280,#bitfinex,unknown wallet,transferred,False,False


In [16]:
wtoe=df[(df['Source'].str.contains("unknown",na=False))]
wtoe

,Datetime,Tweet Id,Text,Username,kw_check,BTC,USD,Source,Destination,transferred,scam,stolen
29,2021-10-17 19:44:48+00:00,1449823730444681220,🚨 🚨 🚨 499 #BTC (30215276 ) transferred from u...,whale_alert,BTC,499,30215276,unknown wallet,#coinbase,transferred,False,False
33,2021-10-17 14:39:50+00:00,1449746983208488967,🚨 🚨 🚨 565 #BTC (34505107 ) transferred from u...,whale_alert,BTC,565,34505107,unknown wallet,#renrenbit,transferred,False,False
88,2021-10-16 05:33:55+00:00,1449247213134229517,🚨 🚨 🚨 🚨 🚨 829 #BTC (51559960 ) transferred fr...,whale_alert,BTC,829,51559960,unknown wallet,#poloniex,transferred,False,False
89,2021-10-16 04:33:02+00:00,1449231891115225095,🚨 🚨 360 #BTC (22408806 ) transferred from unk...,whale_alert,BTC,360,22408806,unknown wallet,#binance,transferred,False,False
96,2021-10-15 23:26:51+00:00,1449154834951806978,🚨 🚨 🚨 🚨 700 #BTC (42773593 ) transferred from...,whale_alert,BTC,700,42773593,unknown wallet,cryptocom,transferred,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
52652,2018-09-21 05:06:41+00:00,1043003561271910404,949 #BTC (6221999 ) transferred from Unknown w...,whale_alert,BTC,949,6221999,unknown wallet,#bitfinex,transferred,False,False
52653,2018-09-21 05:06:41+00:00,1043003560311414786,1000 #BTC (6555335 ) transferred from Unknown ...,whale_alert,BTC,1000,6555335,unknown wallet,#kraken,transferred,False,False
52655,2018-09-21 02:32:07+00:00,1042964663837892613,1500 #BTC (9769703 ) transferred from Unknown ...,whale_alert,BTC,1500,9769703,unknown wallet,#bitfinex,transferred,False,False
52662,2018-09-20 22:23:49+00:00,1042902173942784000,1229 #BTC (7969815 ) transferred from Unknown ...,whale_alert,BTC,1229,7969815,unknown wallet,#bitfinex,transferred,False,False


In [17]:
etow=df[(df['Destination'].str.contains("unknown",na=False))]
etow

,Datetime,Tweet Id,Text,Username,kw_check,BTC,USD,Source,Destination,transferred,scam,stolen
25,2021-10-17 21:34:18+00:00,1449851285948276743,🚨 🚨 🚨 645 #BTC (38674244 ) transferred from #...,whale_alert,BTC,645,38674244,#coinbase,unknown wallet,transferred,False,False
104,2021-10-15 20:59:41+00:00,1449117801663377409,🚨 🚨 341 #BTC (21397789 ) transferred from #Co...,whale_alert,BTC,341,21397789,#coinbase,unknown wallet,transferred,False,False
129,2021-10-15 17:59:04+00:00,1449072344379576324,🚨 🚨 🚨 500 #BTC (30861741 ) transferred from #...,whale_alert,BTC,500,30861741,#xapo,unknown wallet,transferred,False,False
134,2021-10-15 17:11:18+00:00,1449060326650023936,🚨 🚨 🚨 🚨 🚨 881 #BTC (54535823 ) transferred fr...,whale_alert,BTC,881,54535823,#coinbase,unknown wallet,transferred,False,False
218,2021-10-14 19:10:30+00:00,1448727935301656580,🚨 🚨 🚨 🚨 839 #BTC (48322579 ) transferred from...,whale_alert,BTC,839,48322579,#coinbase,unknown wallet,transferred,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
52673,2018-09-19 16:54:53+00:00,1042457007926718464,928 #BTC (5891728 ) transferred from #Bitfinex...,whale_alert,BTC,928,5891728,#bitfinex,unknown wallet,transferred,False,False
52674,2018-09-19 09:04:06+00:00,1042338533606912000,955 #BTC (6083566 ) transferred from #Bitfinex...,whale_alert,BTC,955,6083566,#bitfinex,unknown wallet,transferred,False,False
52678,2018-09-19 04:45:51+00:00,1042273540760850434,946 #BTC (6012967 ) transferred from #Bitfinex...,whale_alert,BTC,946,6012967,#bitfinex,unknown wallet,transferred,False,False
52682,2018-09-18 09:15:51+00:00,1041979103442812928,997 #BTC (6269280 ) transferred from #Bitfinex...,whale_alert,BTC,997,6269280,#bitfinex,unknown wallet,transferred,False,False


In [18]:
#df['Source'].value_counts()

In [19]:
#df['Destination'].value_counts()

In [20]:
etow['Destination'].value_counts()

unknown wallet                                                                                           4236
unknown wallet                                                                                           1336
unknown wallet                                                                                            342
unknown walletℹ️ the coins in this transaction were mined in the first month of bitcoin's existence.        1
Name: Destination, dtype: int64

In [21]:
#wtoe['Destination'].value_counts()

In [22]:
wtoe['Source'].value_counts()

 unknown wallet     4263
Name: Source, dtype: int64

In [23]:
wtoe['Destination']=wtoe['Destination'].str.rstrip()
wtoe=wtoe[wtoe['Destination']!="unknown wallet"]

<ipython-input-23-fb39f0fd0b7e>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
etow['Source']=etow['Source'].str.rstrip()
etow['Source']=etow['Source'].str.lstrip()
etow[etow['Destination']!="unknown walletℹ️ the coins in this transaction were mined in the first month of bitcoin's existence. "]
etow=etow[etow['Source']!="unknown wallet"]

<ipython-input-24-795d0bd1837b>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-24-795d0bd1837b>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
etow.to_csv("cleaned_twitter_etow.csv",index=False)
wtoe.to_csv("cleaned_twitter_wtoe.csv",index=False)